<a href="https://colab.research.google.com/github/lyj12h/ai_m/blob/main/practice/0927practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setup

In [1]:
!wget https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_03/data/long_data.csv
!wget https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_03/data/wide_data.csv
!wget https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_03/data/dirty_data.csv

--2023-09-30 01:11:07--  https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_03/data/long_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5494 (5.4K) [text/plain]
Saving to: ‘long_data.csv’

long_data.csv       100%[===================>]   5.37K  --.-KB/s    in 0s      

2023-09-30 01:11:07 (64.2 MB/s) - ‘long_data.csv’ saved [5494/5494]

--2023-09-30 01:11:07--  https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_03/data/wide_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connect

In [64]:
import numpy as np
import pandas as pd

# Task 1

In [112]:
#rename : 이름 바꿔주기
#pd.to_datetime : 날짜 형식 바꿔주기
long=pd.read_csv('long_data.csv',
            usecols=['datatype','date','value']).rename(
    columns={'value': 'temp_C'}
).assign(temp_F= lambda x:(x.temp_C * 9/5) + 32,
         date=lambda x: pd.to_datetime(x.date)
            )
long

,datatype,date,temp_C,temp_F
0,TMAX,2018-10-01,21.1,69.98
1,TMIN,2018-10-01,8.9,48.02
2,TOBS,2018-10-01,13.9,57.02
3,TMAX,2018-10-02,23.9,75.02
4,TMIN,2018-10-02,13.9,57.02
...,...,...,...,...
88,TMIN,2018-10-30,2.2,35.96
89,TOBS,2018-10-30,5.0,41.00
90,TMAX,2018-10-31,12.2,53.96
91,TMIN,2018-10-31,0.0,32.00


## long -> wide

In [66]:
#wide data
pivoted_df = long.pivot(
    index='date', columns='datatype', values='temp_C'
)
pivoted_df.head()

datatype,TMAX,TMIN,TOBS
date,,,
2018-10-01,21.1,8.9,13.9
2018-10-02,23.9,13.9,17.2
2018-10-03,25.0,15.6,16.1
2018-10-04,22.8,11.7,11.7
2018-10-05,23.3,11.7,18.9


In [67]:
# set_index를 통해 인덱스를 설정하고 unstack 사용하기
unstacked_df = long.set_index(['date', 'datatype']).unstack()
unstacked_df.head()

temp_C             temp_F              
datatype     TMAX  TMIN  TOBS   TMAX   TMIN   TOBS
date                                              
2018-10-01   21.1   8.9  13.9  69.98  48.02  57.02
2018-10-02   23.9  13.9  17.2  75.02  57.02  62.96
2018-10-03   25.0  15.6  16.1  77.00  60.08  60.98
2018-10-04   22.8  11.7  11.7  73.04  53.06  53.06
2018-10-05   23.3  11.7  18.9  73.94  53.06  66.02

In [68]:
# 딕셔너리 형태로 ? 넣기 - append
extra_data = long.append([{
    'datatype': 'TAVG',
    'date': '2018-10-01',
    'temp_C': 10,
    'temp_F': 50
}]).set_index(['date', 'datatype']).sort_index()

extra_data['2018-10-01':'2018-10-02']

<ipython-input-68-11d07c5705e6>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_data = long.append([{
<ipython-input-68-11d07c5705e6>:7: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  }]).set_index(['date', 'datatype']).sort_index()


temp_C  temp_F
date       datatype                
2018-10-01 TAVG        10.0   50.00
           TMAX        21.1   69.98
           TMIN         8.9   48.02
           TOBS        13.9   57.02
2018-10-02 TMAX        23.9   75.02
           TMIN        13.9   57.02
           TOBS        17.2   62.96

In [69]:
# 새 datafrmae
df3 = pd.DataFrame(data=[['TAVG','2018-10-01 00:00:00',10,50],['TAVG','2018-10-02 00:00:00',10,50]], columns=['datatype','date','temp_C','temp_F'])
df3

,datatype,date,temp_C,temp_F
0,TAVG,2018-10-01 00:00:00,10,50
1,TAVG,2018-10-02 00:00:00,10,50


In [70]:
# dataframe끼리 합치기
extra=pd.concat([long,df3],ignore_index=True).assign(date=lambda x: pd.to_datetime(x.date)).set_index(['date', 'datatype']).sort_index()

extra['2018-10-01':'2018-10-02']

temp_C  temp_F
date       datatype                
2018-10-01 TAVG        10.0   50.00
           TMAX        21.1   69.98
           TMIN         8.9   48.02
           TOBS        13.9   57.02
2018-10-02 TAVG        10.0   50.00
           TMAX        23.9   75.02
           TMIN        13.9   57.02
           TOBS        17.2   62.96

In [71]:
# unstack을 사용해서 wide 로 바꾸기 + Nan값에 일정 값 입력
new_extra=extra.unstack(fill_value=-40).head()
new_extra

temp_C                   temp_F                     
datatype     TAVG  TMAX  TMIN  TOBS   TAVG   TMAX   TMIN   TOBS
date                                                           
2018-10-01   10.0  21.1   8.9  13.9   50.0  69.98  48.02  57.02
2018-10-02   10.0  23.9  13.9  17.2   50.0  75.02  57.02  62.96
2018-10-03  -40.0  25.0  15.6  16.1  -40.0  77.00  60.08  60.98
2018-10-04  -40.0  22.8  11.7  11.7  -40.0  73.04  53.06  53.06
2018-10-05  -40.0  23.3  11.7  18.9  -40.0  73.94  53.06  66.02

##wide->long

In [72]:
wide=pd.read_csv('wide_data.csv')
wide.head()

,date,TMAX,TMIN,TOBS
0,2018-10-01,21.1,8.9,13.9
1,2018-10-02,23.9,13.9,17.2
2,2018-10-03,25.0,15.6,16.1
3,2018-10-04,22.8,11.7,11.7
4,2018-10-05,23.3,11.7,18.9


In [73]:
# melt
## var : variable
melted=wide.melt(
    id_vars='date',
    value_vars=['TMAX','TMIN','TOBS'],
    value_name='temp_C',
    var_name='measurement'
)
melted.head()

,date,measurement,temp_C
0,2018-10-01,TMAX,21.1
1,2018-10-02,TMAX,23.9
2,2018-10-03,TMAX,25.0
3,2018-10-04,TMAX,22.8
4,2018-10-05,TMAX,23.3


In [74]:
# stack
## index 설정 후 인덱스 기준으로 값을 쌓아라 !
stacked=wide.set_index('date')
stacked.head()

,TMAX,TMIN,TOBS
date,,,
2018-10-01,21.1,8.9,13.9
2018-10-02,23.9,13.9,17.2
2018-10-03,25.0,15.6,16.1
2018-10-04,22.8,11.7,11.7
2018-10-05,23.3,11.7,18.9


In [75]:
# to_frame(값의 이름)
## to_frame을 해야지 데이터프레임 형태로 보여줌
stacked.stack().to_frame('values')

values
date                   
2018-10-01 TMAX    21.1
           TMIN     8.9
           TOBS    13.9
2018-10-02 TMAX    23.9
           TMIN    13.9
...                 ...
2018-10-30 TMIN     2.2
           TOBS     5.0
2018-10-31 TMAX    12.2
           TMIN     0.0
           TOBS     0.0

[93 rows x 1 columns]

In [113]:
stacked.index.names

FrozenList(['date'])

#Task2

In [115]:
##데이터 확인작업
df = pd.read_csv('dirty_data.csv')
df.head()
#station 이름이 없는 부분이나 -inf, Nan 값 등 문제가 있는 부분이 많음, index를 봤을 때 0~3까지 중복된 데이터로 예상

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
0,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
3,2018-01-02T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
4,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False


In [116]:
# 기초통계량 보기
df.describe()

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:4527: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF
count,765.000000,577.000000,577.0,765.000000,765.000000,398.000000,11.000000
mean,5.360392,4.202773,NaN,2649.175294,-15.914379,8.632161,16.290909
std,10.002138,25.086077,NaN,2744.156281,24.242849,9.815054,9.489832
min,0.000000,0.000000,-inf,-11.700000,-40.000000,-16.100000,1.800000
25%,0.000000,0.000000,NaN,13.300000,-40.000000,0.150000,8.600000
50%,0.000000,0.000000,NaN,32.800000,-11.100000,8.300000,19.300000
75%,5.800000,0.000000,NaN,5505.000000,6.700000,18.300000,24.900000
max,61.700000,229.000000,inf,5505.000000,23.900000,26.100000,28.700000


In [117]:
#데이터 정보 불러오기 - obs가 차이가 있으면 결측치가 있는거
df.info()
##snow, snwd, tobs, wesf에서 결측치가 있음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765 entries, 0 to 764
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               765 non-null    object 
 1   station            765 non-null    object 
 2   PRCP               765 non-null    float64
 3   SNOW               577 non-null    float64
 4   SNWD               577 non-null    float64
 5   TMAX               765 non-null    float64
 6   TMIN               765 non-null    float64
 7   TOBS               398 non-null    float64
 8   WESF               11 non-null     float64
 9   inclement_weather  408 non-null    object 
dtypes: float64(7), object(3)
memory usage: 59.9+ KB


In [118]:
# 앞의 정보를 바탕으로
## 결측 데이터 추출 - isna 활용
### SNOW에 isna가 있거나 SNWD에 nan이 있거나 .. 적어도 하나에 있으면 contain_nulls에 df 형식으로
contain_nulls = df[
    df.SNOW.isna() | df.SNWD.isna() | df.TOBS.isna()
    | df.WESF.isna() | df.inclement_weather.isna()
]
contain_nulls.shape[0] # 행의 갯수 확인 cf.shape[0]하면 열의 갯수

765

In [119]:
contain_nulls.head(10)

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
0,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
3,2018-01-02T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
4,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
5,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
6,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
7,2018-01-04T00:00:00,?,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
8,2018-01-04T00:00:00,?,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
9,2018-01-05T00:00:00,?,0.3,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN


In [120]:
# 모든 데이터에서 -무한대, 무한대 값 체크
## 무한대값 갯수 세기
def get_inf_count(df):
    return {
        col: df[df[col].isin([np.inf, -np.inf])].shape[0] for col in df.columns
    }
#columns로 루프를 돌린다
get_inf_count(df)

{'date': 0,
 'station': 0,
 'PRCP': 0,
 'SNOW': 0,
 'SNWD': 577,
 'TMAX': 0,
 'TMIN': 0,
 'TOBS': 0,
 'WESF': 0,
 'inclement_weather': 0}

In [121]:
#depth와 가장 관련이 높은 snow 값 통계를 이용해서 inf,-inf 값의 경향 ? 확인

pd.DataFrame(
    {
        'np.inf snow depth': df[df.SNWD==np.inf].SNOW.describe(),
        '-np.inf snow depth': df[df.SNWD==-np.inf].SNOW.describe()
    }
).T

### 확인헸더니 np.inf의 snow 최대값은 229 - 얘를 넘으면 안되겠지 하는 수정
### -np.inf는 0 눈이 안내린 경우

,count,mean,std,min,25%,50%,75%,max
np.inf snow depth,24.0,101.041667,74.498018,13.0,25.0,120.5,152.0,229.0
-np.inf snow depth,553.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [122]:
# 역이름에 ? 확인
df.describe()

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:4527: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF
count,765.000000,577.000000,577.0,765.000000,765.000000,398.000000,11.000000
mean,5.360392,4.202773,NaN,2649.175294,-15.914379,8.632161,16.290909
std,10.002138,25.086077,NaN,2744.156281,24.242849,9.815054,9.489832
min,0.000000,0.000000,-inf,-11.700000,-40.000000,-16.100000,1.800000
25%,0.000000,0.000000,NaN,13.300000,-40.000000,0.150000,8.600000
50%,0.000000,0.000000,NaN,32.800000,-11.100000,8.300000,19.300000
75%,5.800000,0.000000,NaN,5505.000000,6.700000,18.300000,24.900000
max,61.700000,229.000000,inf,5505.000000,23.900000,26.100000,28.700000


In [123]:
df.describe(include='object')

#station에 unique값이 2개밖에 없음

,date,station,inclement_weather
count,765,765,408
unique,324,2,2
top,2018-07-05T00:00:00,GHCND:USC00280907,False
freq,8,398,384


In [124]:
#중복된 데이터 제거
df[df.duplicated()] # 중복된 항목 확인

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
5,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
6,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
8,2018-01-04T00:00:00,?,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
...,...,...,...,...,...,...,...,...,...,...
756,2018-12-28T00:00:00,?,11.4,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN
758,2018-12-29T00:00:00,?,21.3,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN
761,2018-12-31T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,3.3,-3.3,-2.8,NaN,False
762,2018-12-31T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,3.3,-3.3,-2.8,NaN,False


In [125]:
df.WESF.notna()

0      False
1      False
2      False
3      False
4      False
       ...  
760    False
761    False
762    False
763    False
764    False
Name: WESF, Length: 765, dtype: bool

In [126]:
df[df.duplicated(keep=False)].head()

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
0,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
4,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
5,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False


In [127]:
df[df.duplicated(keep='first')].head()

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
5,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
6,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
8,2018-01-04T00:00:00,?,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True


In [128]:
# column을 지정해 중복된 항목 확인
df[df.duplicated(['date', 'station'])].shape[0]

284

### 중복된 항목 다루기

In [129]:
df[df.WESF.notna()].station.unique()

array(['?'], dtype=object)

In [130]:
df.WESF.notna()

0      False
1      False
2      False
3      False
4      False
       ...  
760    False
761    False
762    False
763    False
764    False
Name: WESF, Length: 765, dtype: bool

In [131]:
# 1. make the date a datetime
df.date = pd.to_datetime(df.date)

# 2. save this information for later
station_qm_wesf = df[df.station == '?'].drop_duplicates('date').set_index('date').WESF

# 3. sort ? to the bottom
df.sort_values('station', ascending=False, inplace=True)

# 4. drop duplicates based on the date column keeping the first occurrence
# which will be the valid station if it has data
df_deduped = df.drop_duplicates('date')

# 5. remove the station column because we are done with it
df_deduped = df_deduped.drop(columns='station').set_index('date').sort_index()

# 6. take valid station's WESF and fall back on station ? if it is null
df_deduped = df_deduped.assign(
    WESF=lambda x: x.WESF.combine_first(station_qm_wesf)
)

df_deduped.shape

(324, 8)

In [132]:
df.date=pd.to_datetime(df.date)

In [133]:
station_qm_wesf2=df[df.station== '?'].drop_duplicates('date').set_index('date')
station_qm_wesf2

,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,,
2018-09-04,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2018-09-10,?,23.9,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN
2018-09-05,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2018-09-01,?,5.8,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN
2018-09-02,?,0.3,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2018-06-12,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2018-06-11,?,1.3,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN
2018-06-10,?,0.5,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN


In [134]:
## station이 ?인 값에서 중복된 값을 제거한 WESF 정보 추출
station_qm_wesf3=df[df.station== '?'].drop_duplicates('date').set_index('date').WESF
station_qm_wesf3

date
2018-09-04   NaN
2018-09-10   NaN
2018-09-05   NaN
2018-09-01   NaN
2018-09-02   NaN
              ..
2018-06-12   NaN
2018-06-11   NaN
2018-06-10   NaN
2018-06-09   NaN
2018-06-08   NaN
Name: WESF, Length: 232, dtype: float64

In [135]:
df[df.station != '?']

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
510,2018-08-31,GHCND:USC00280907,0.0,0.0,-inf,33.3,21.1,21.7,NaN,False
642,2018-11-02,GHCND:USC00280907,1.0,0.0,-inf,21.1,8.9,19.4,NaN,False
538,2018-09-12,GHCND:USC00280907,1.3,0.0,-inf,22.8,15.6,21.1,NaN,False
537,2018-09-12,GHCND:USC00280907,1.3,0.0,-inf,22.8,15.6,21.1,NaN,False
636,2018-10-29,GHCND:USC00280907,3.3,0.0,-inf,10.6,6.7,8.3,NaN,False
...,...,...,...,...,...,...,...,...,...,...
708,2018-12-05,GHCND:USC00280907,0.0,0.0,-inf,3.9,-6.7,-6.7,NaN,False
136,2018-03-08,GHCND:USC00280907,48.8,NaN,NaN,1.1,-0.6,1.1,NaN,False
135,2018-03-08,GHCND:USC00280907,48.8,NaN,NaN,1.1,-0.6,1.1,NaN,False
150,2018-03-16,GHCND:USC00280907,0.0,0.0,-inf,6.7,-1.1,-0.6,NaN,False


In [136]:
df_deduped = df.drop_duplicates('date')
df_deduped

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
510,2018-08-31,GHCND:USC00280907,0.0,0.0,-inf,33.3,21.1,21.7,NaN,False
642,2018-11-02,GHCND:USC00280907,1.0,0.0,-inf,21.1,8.9,19.4,NaN,False
538,2018-09-12,GHCND:USC00280907,1.3,0.0,-inf,22.8,15.6,21.1,NaN,False
636,2018-10-29,GHCND:USC00280907,3.3,0.0,-inf,10.6,6.7,8.3,NaN,False
282,2018-05-15,GHCND:USC00280907,0.5,0.0,-inf,23.3,10.6,17.8,NaN,False
...,...,...,...,...,...,...,...,...,...,...
350,2018-06-15,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
328,2018-06-07,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
344,2018-06-12,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
342,2018-06-11,?,1.3,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN


In [137]:
#station이 결과에 영향을 안미친다고 가정하고 station 항목 제거
df_deduped.drop(columns='station')

/usr/local/lib/python3.10/dist-packages/IPython/lib/pretty.py:700: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/usr/local/lib/python3.10/dist-packages/google/colab/_interactive_table_hint_button.py:178: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_html=dataframe._repr_html_(),  # pylint: disable=protected-access


,date,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
510,2018-08-31,0.0,0.0,-inf,33.3,21.1,21.7,NaN,False
642,2018-11-02,1.0,0.0,-inf,21.1,8.9,19.4,NaN,False
538,2018-09-12,1.3,0.0,-inf,22.8,15.6,21.1,NaN,False
636,2018-10-29,3.3,0.0,-inf,10.6,6.7,8.3,NaN,False
282,2018-05-15,0.5,0.0,-inf,23.3,10.6,17.8,NaN,False
...,...,...,...,...,...,...,...,...,...
350,2018-06-15,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
328,2018-06-07,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
344,2018-06-12,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
342,2018-06-11,1.3,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN


In [138]:
#index를 날짜로 바꿈
df_deduped=df_deduped.drop(columns='station').set_index('date')
df_deduped

/usr/local/lib/python3.10/dist-packages/IPython/lib/pretty.py:700: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/usr/local/lib/python3.10/dist-packages/google/colab/_interactive_table_hint_button.py:178: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_html=dataframe._repr_html_(),  # pylint: disable=protected-access


,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-08-31,0.0,0.0,-inf,33.3,21.1,21.7,NaN,False
2018-11-02,1.0,0.0,-inf,21.1,8.9,19.4,NaN,False
2018-09-12,1.3,0.0,-inf,22.8,15.6,21.1,NaN,False
2018-10-29,3.3,0.0,-inf,10.6,6.7,8.3,NaN,False
2018-05-15,0.5,0.0,-inf,23.3,10.6,17.8,NaN,False
...,...,...,...,...,...,...,...,...
2018-06-15,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2018-06-07,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2018-06-12,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN


In [139]:
#날짜 순서대로 배열
df_deduped=df_deduped.sort_index()
df_deduped

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
2018-01-04,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,NaN,True
...,...,...,...,...,...,...,...,...
2018-12-27,0.0,0.0,-inf,5.6,-2.2,-1.1,NaN,False
2018-12-28,11.7,0.0,-inf,6.1,-1.7,5.0,NaN,False
2018-12-29,21.3,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN


In [140]:
df_deduped=df_deduped.assign(
    WESF=lambda x: x.WESF.combine_first(station_qm_wesf3))
df_deduped

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
2018-01-04,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,NaN,True
...,...,...,...,...,...,...,...,...
2018-12-27,0.0,0.0,-inf,5.6,-2.2,-1.1,NaN,False
2018-12-28,11.7,0.0,-inf,6.1,-1.7,5.0,NaN,False
2018-12-29,21.3,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN


###결측값 처리

In [141]:
#how='all' : nan값이 있는 모든 row 항목을 제거
##subset으로 어떤 column에 nan이 있을 때인지를 지정을 해줘야함
### (324,8) > (247,8)
df_deduped.dropna(
    how='all', subset=['inclement_weather', 'SNOW', 'SNWD']
).shape

(293, 8)

In [142]:
#na 값에 0 채우기
##loc를 통해 위치를 지정
df_deduped.loc[:,'WESF'].fillna(0, inplace=True)
df_deduped.head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,5505.0,-40.0,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,0.0,True


In [143]:
#TMAX,TMIN의 nan 값에 적절한 값 입력
df_deduped = df_deduped.assign(
    TMAX=lambda x: x.TMAX.replace(5505, np.nan),
    TMIN=lambda x: x.TMIN.replace(-40, np.nan),
)

In [144]:
#ffill로 할 경우 결측값을 바로 위 값과 동일하게 변경
#bfill : 바로 아래 값과 동일하게 변경
df_deduped.assign(
    TMAX=lambda x: x.TMAX.fillna(method='ffill'),
    TMIN=lambda x: x.TMIN.fillna(method='ffill')
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,inf,-4.4,-13.9,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,0.0,True


In [145]:
#inf값을 number이면서 엄청 작은 값으로 바꿔줌
df_deduped.assign(
    SNWD=lambda x: np.nan_to_num(x.SNWD)
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-1.797693e+308,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-1.797693e+308,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-1.797693e+308,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,1.797693e+308,NaN,NaN,NaN,19.3,True
2018-01-05,14.2,127.0,1.797693e+308,-4.4,-13.9,-13.9,0.0,True


In [146]:
# clip : cap values at a specific minimum and/or maximum threshold
df_deduped.assign(
    SNWD=lambda x: x.SNWD.clip(0, x.SNOW)
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,0.0,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,0.0,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,229.0,NaN,NaN,NaN,19.3,True
2018-01-05,14.2,127.0,127.0,-4.4,-13.9,-13.9,0.0,True


In [147]:
#조건을 줘서 결측값 채우는 방법
df_deduped.assign(
    TMAX=lambda x: x.TMAX.fillna(x.TMAX.median()),
    TMIN=lambda x: x.TMIN.fillna(x.TMIN.median()),
    # average of TMAX and TMIN
    TOBS=lambda x: x.TOBS.fillna((x.TMAX + x.TMIN) / 2)
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,14.4,5.6,10.0,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,inf,14.4,5.6,10.0,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,0.0,True


In [148]:
df_deduped.apply(
    # rolling calculations will be covered in chapter 4, this is a rolling 7-day median
    # we set min_periods (# of periods required for calculation) to 0 so we always get a result
    lambda x: x.fillna(x.rolling(7, min_periods=0).median())
).head(10)

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.30,-16.1,-12.20,0.0,False
2018-01-03,0.0,0.0,-inf,-4.40,-13.9,-13.30,0.0,False
2018-01-04,20.6,229.0,inf,-6.35,-15.0,-12.75,19.3,True
2018-01-05,14.2,127.0,inf,-4.40,-13.9,-13.90,0.0,True
2018-01-06,0.0,0.0,-inf,-10.00,-15.6,-15.00,0.0,False
2018-01-07,0.0,0.0,-inf,-11.70,-17.2,-16.10,0.0,False
2018-01-08,0.0,0.0,-inf,-7.80,-16.7,-8.30,0.0,False
2018-01-10,0.0,0.0,-inf,5.00,-7.8,-7.80,0.0,False


In [149]:
df_deduped\
    .reindex(pd.date_range('2018-01-01', '2018-12-31', freq='D'))\
    .apply(lambda x: x.interpolate())\
    .head(10)

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
2018-01-01,0.0,0.0,-inf,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.10,-12.20,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.90,-13.30,0.0,False
2018-01-04,20.6,229.0,inf,-4.4,-13.90,-13.60,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.90,-13.90,0.0,True
2018-01-06,0.0,0.0,-inf,-10.0,-15.60,-15.00,0.0,False
2018-01-07,0.0,0.0,-inf,-11.7,-17.20,-16.10,0.0,False
2018-01-08,0.0,0.0,-inf,-7.8,-16.70,-8.30,0.0,False
2018-01-09,0.0,0.0,-inf,-1.4,-12.25,-8.05,0.0,NaN
2018-01-10,0.0,0.0,-inf,5.0,-7.80,-7.80,0.0,False
